<h2>3D stack to 2D MIP - Single image - Marker+ based on average intensity</h2>

The following notebook is able to process a 3D stack (.czi or .nd2 files) into a MIP and allows you to:

1. Inspect your images in Napari.
2. Define ROIs if needed.
3. Read previously defined ROIs, if not present, full image is analyzed.
4. Read previously predicted nuclei labels, if not present, generates them.
5. Plot cytoplasmic and nuclear average intensities of all markers in all ROIs to later on set a threshold for classification.
6. Extract numbers of cells positive for all marker based on signal average intensity within the nuclear or cytoplasmic compartments (using a user-defined min-max range).
6. Display positive cells in Napari.
7. Extract and save per label per ROI per marker data in a .csv file (filename_per_label_avg_int.csv).
8. Extract and save number of positive cells in a .csv file (SP_marker_+_label_avg_int.csv).

In [1]:
from pathlib import Path
import tifffile
import napari
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
from skimage.measure import regionprops_table
import plotly.express as px
from utils_cellpose import list_images, read_image, save_rois, simulate_cytoplasm, segment_nuclei_2d

c:\Users\adiez_cmic\miniforge3\envs\brain_nuc_cellpose\Lib\site-packages\cellpose\resnet_torch.py:276: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(

<h3>Define the directory for your images (.nd2 or .czi files) and cell marker info</h3>

In [2]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path("../raw_data/test_data")

# Define the channels you want to analyze using the following structure:
# markers = [(channel_name, channel_nr, cellular_location),(..., ..., ...)]
# Remember in Python one starts counting from 0, so your first channel will be 0
# markers = [("ki67", 0, "nucleus"), ("neun", 1, "nucleus"), ("calbindin", 2, "cytoplasm")]
markers = [("ki67", 0, "nucleus"), ("neun", 1, "nucleus"), ("calbindin", 2, "cytoplasm")]

# Iterate through the .czi and .nd2 files in the raw_data directory
images = list_images(directory_path)

images

['..\\raw_data\\test_data\\HI1_CONTRA_M8_S6_TR1.czi',
 '..\\raw_data\\test_data\\HI1_CONTRA_M8_S6_TR2.czi',
 '..\\raw_data\\test_data\\HI1_CONTRA_M8_S7_TR1.czi',
 '..\\raw_data\\test_data\\HI1_CONTRA_M8_S7_TR2.czi',
 '..\\raw_data\\test_data\\HI1_IPSI_M8_S6_TR1.czi',
 '..\\raw_data\\test_data\\HI1_IPSI_M8_S6_TR2.czi',
 '..\\raw_data\\test_data\\HI1_IPSI_M8_S7_TR1.czi',
 '..\\raw_data\\test_data\\HI2_CONTRA_M10_S10_TR1.czi',
 '..\\raw_data\\test_data\\HI2_CONTRA_M10_S10_TR2.czi',
 '..\\raw_data\\test_data\\HI2_IPSI_M10_S10_TR1.czi',
 '..\\raw_data\\test_data\\HI2_IPSI_M10_S10_TR2.czi',
 '..\\raw_data\\test_data\\HI3_CONTRA_M11_S10_TR1.czi',
 '..\\raw_data\\test_data\\HI3_CONTRA_M11_S10_TR2.czi',
 '..\\raw_data\\test_data\\HI3_IPSI_M11_S10_TR1.czi',
 '..\\raw_data\\test_data\\HI3_IPSI_M11_S10_TR2.czi',
 '..\\raw_data\\test_data\\SHAM1_CONTRA_M6_S11_TR1.czi',
 '..\\raw_data\\test_data\\SHAM1_CONTRA_M6_S11_TR2.czi',
 '..\\raw_data\\test_data\\SHAM1_CONTRA_M7_S11_TR1.czi',
 '..\\raw_data\\t

<h3>Open each image in the directory</h3>
You can do so by changing the number within the brackets below <code>image = images[0]</code>. Make sure to input the same <code>slicing factor</code> you used when generating nuclei predictions. 

If you have not generated nuclei predictions already, input <code>nuclei_channel</code>, <code>cellpose_nuclei_diameter</code> and <code>gaussian_sigma</code> values.

In [3]:
# Explore each image to analyze (0 defines the first image in the directory)
image = images[0]

# Remember to input the same slicing factor you used when generating the nuclei predictions
slicing_factor = None # Use 2 or 4 for compression (None for lossless)

# Define the nuclei and markers of interest channel order ('Remember in Python one starts counting from zero')
nuclei_channel = 3

# Segmentation type ("2D" or "3D"). 
# 2D takes a z-stack as input, performs MIP (Maximum Intensity Projection) and predicts nuclei from the resulting projection (faster, useful for single layers of cells)
# 3D is more computationally expensive. Predicts 3D nuclear volumes, useful for multilayered structures
segmentation_type = "2D"

# This is a placeholder to later choose from fine-tuned Cellpose models (default nuclei model in Cellpose 3.0)
model_name = "Cellpose"

# Define your nuclei diameter, it speeds up nuclei detection, if unknown leave it as None
cellpose_nuclei_diameter = None

# Define the amount of blur applied to nuclei
# Blurs the mip_nuclei image to even out high intensity foci within the nucleus, the higher the value blurriness increases
# High values help segment sparse nuclei (CA and CTX regions) but as a drawback it merges nuclei entities that are very close together (DG region)
gaussian_sigma = 0 

# Generate maximum intensity projection and extract filename
img_mip, filename = read_image(image, slicing_factor)

# Show image in Napari
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img_mip)



Image analyzed: HI1_CONTRA_M8_S6_TR1
Original Array shape: (4, 14, 3803, 2891)
MIP Array shape: (4, 3803, 2891)


Invalid schema for package 'ome-types', please run 'npe2 validate ome-types' to check for manifest errors.


<Image layer 'img_mip' at 0x1bedfcdecd0>

<h3>Label your regions of interest in Napari and explore the signal of your marker of interest</h3>

Make sure to set <code>n edit dim = 3</code> so the label propagates across all channels. Name your regions of interest as i.e. <code>DG</code>, <code>CA1</code>, <code>CA3</code> or <code>HIPPO</code>. If you do not draw any ROI the entire image will be analyzed.

<video controls>
  <source src="../assets/napari_labels.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

<h3>Save user-defined label ROIs as .tiff files</h3>

In [4]:
save_rois(viewer, directory_path, filename)

No user-defined ROIs have been stored


<h3>Extract average intensities of markers within each defined cell compartment and ROI</h3>

In [5]:
# Construct ROI and nuclei predictions paths from directory_path above
roi_path = directory_path / "ROIs"
nuclei_preds_path =  directory_path / "nuclei_preds" / segmentation_type / model_name

# Extract the experiment name from the data directory path
experiment_id = directory_path.name

# List of subfolder names
try:
    roi_names = [folder.name for folder in roi_path.iterdir() if folder.is_dir()]

except FileNotFoundError:
    roi_names = ["full_image"]
        
print(f"The following regions of interest will be analyzed: {roi_names}")

# Initialize an empty list to hold the extracted dataframes on a per ROI basis
per_roi_props = []

for roi_name in tqdm(roi_names):
    print(f"\nAnalyzing ROI: {roi_name}")

    # Initialize an empty list to hold the extracted dataframes on a per channel basis
    props_list = []

    # Read the user defined ROIs, in case of full image analysis generate a label covering the entire image
    try:
        # Read previously defined ROIs
        user_roi = tifffile.imread(roi_path / roi_name / f"{filename}.tiff")

    except FileNotFoundError:
        # Extract the xy dimensions of the input image 
        img_shape = img_mip.shape
        img_xy_dims = img_shape[-2:]

        # Create a label covering the entire image
        user_roi = np.ones(img_xy_dims).astype(np.uint8)

    # Read previously predicted nuclei labels, if not present generate nuclei predictions and save them
    try:
        # Read the nuclei predictions per ROI
        nuclei_labels = tifffile.imread(nuclei_preds_path / roi_name / f"{filename}.tiff")
        print(f"Pre-computed nuclei labels found for {filename}")
    except FileNotFoundError:
        print(f"Generating nuclei labels for {filename}")
        # Slice the nuclei stack
        nuclei_img = img_mip[nuclei_channel, :, :]

        # We will create a mask where roi is greater than or equal to 1
        mask = (user_roi >= 1).astype(np.uint8)

        # Apply the mask to nuclei_img and marker_img, setting all other pixels to 0
        masked_nuclei_img = np.where(mask, nuclei_img, 0)

        # Segment nuclei and return labels
        nuclei_labels = segment_nuclei_2d(masked_nuclei_img, gaussian_sigma, cellpose_nuclei_diameter)

        # Save nuclei labels as .tiff files to reuse them later
        # Create nuclei_predictions directory if it does not exist
        try:
            os.makedirs(nuclei_preds_path / roi_name)
        except FileExistsError:
            pass

        # Construct path to store
        path_to_store = nuclei_preds_path / roi_name / f"{filename}.tiff"

        # Save mask (binary image)
        tifffile.imwrite(path_to_store, nuclei_labels)

    # Add the predicted nuclei as labels into Napari
    viewer.add_labels(nuclei_labels, name=f"{roi_name}_nuclei")

    # Add the ROIs as labels into Napari
    viewer.add_labels(user_roi, name=f"{roi_name}_ROI", opacity=0.4)

    # Create a dictionary containing all image descriptors
    descriptor_dict = {
                "filename": filename,
                "ROI": roi_name,
                }

    # Loop through each channel and extract the average intensity within either nuclei or cytoplasmic regions
    for tuple in markers:

        channel_name = tuple[0]
        ch_nr = tuple[1]
        location = tuple[2]

        print(f"Analyzing channel: {channel_name}")

        if location == "cytoplasm":
            print(f"Generating cytoplasm labels for: {channel_name}")
            # Simulate a cytoplasm by dilating the nuclei and substracting the nuclei mask afterwards
            cytoplasm_labels = simulate_cytoplasm(nuclei_labels, dilation_radius = 2, erosion_radius = 0)

            # Add the predicted nuclei as labels into Napari
            viewer.add_labels(cytoplasm_labels, name=f"{roi_name}_cytoplasm")

            # Extract intensity information from each marker channel
            props = regionprops_table(label_image=cytoplasm_labels,
                                intensity_image=img_mip[ch_nr],
                                properties=["label", "intensity_mean"])
        
        elif location == "nucleus":

            # Extract intensity information from each marker channel
            props = regionprops_table(label_image=nuclei_labels,
                                intensity_image=img_mip[ch_nr],
                                properties=["label", "intensity_mean"])
        

        # Convert to dataframe
        props_df = pd.DataFrame(props)

        # Rename intensity_mean column to indicate the specific image
        props_df.rename(columns={"intensity_mean": f"{location}_{channel_name}_avg_int"}, inplace=True)

        # Append each props_df to props_list
        props_list.append(props_df)

        # Initialize the df with the first df in the list
        props_df = props_list[0]
        # Start looping from the second df in the list
        for df in props_list[1:]:
            props_df = props_df.merge(df, on="label")

    # Add each key-value pair from descriptor_dict to props_df at the specified position
    insertion_position = 0    
    for key, value in descriptor_dict.items():
        props_df.insert(insertion_position, key, value)
        insertion_position += 1  # Increment position to maintain the order of keys in descriptor_dict

    # Append each props_df to props_list
    per_roi_props.append(props_df)

final_df = pd.concat(per_roi_props, ignore_index=True)

final_df

The following regions of interest will be analyzed: ['CA', 'DG']


  0%|          | 0/2 [00:00<?, ?it/s]


Analyzing ROI: CA
Pre-computed nuclei labels found for HI1_CONTRA_M8_S6_TR1
Analyzing channel: ki67
Analyzing channel: neun
Analyzing channel: calbindin
Generating cytoplasm labels for: calbindin


 50%|█████     | 1/2 [00:01<00:01,  1.13s/it]


Analyzing ROI: DG
Pre-computed nuclei labels found for HI1_CONTRA_M8_S6_TR1
Analyzing channel: ki67
Analyzing channel: neun
Analyzing channel: calbindin
Generating cytoplasm labels for: calbindin


100%|██████████| 2/2 [00:01<00:00,  1.03it/s]


,filename,ROI,label,nucleus_ki67_avg_int,nucleus_neun_avg_int,cytoplasm_calbindin_avg_int
0,HI1_CONTRA_M8_S6_TR1,CA,1,86.300962,97.016346,15.955224
1,HI1_CONTRA_M8_S6_TR1,CA,2,65.091667,16.241667,8.528302
2,HI1_CONTRA_M8_S6_TR1,CA,3,64.604167,39.833333,9.206897
3,HI1_CONTRA_M8_S6_TR1,CA,4,86.377637,90.877637,11.417582
4,HI1_CONTRA_M8_S6_TR1,CA,5,108.471429,46.031429,16.793893
...,...,...,...,...,...,...
2849,HI1_CONTRA_M8_S6_TR1,DG,1580,94.090411,14.279452,14.977143
2850,HI1_CONTRA_M8_S6_TR1,DG,1581,111.038462,155.445344,30.577236
2851,HI1_CONTRA_M8_S6_TR1,DG,1582,135.163934,23.846995,15.620370
2852,HI1_CONTRA_M8_S6_TR1,DG,1583,97.029748,39.167048,20.030303


<h3>Save per label per ROI per marker data</h3>

In [6]:
# Create a 'results' folder in the root directory
results_folder = Path("results") / experiment_id / segmentation_type / model_name

try:
    os.makedirs(results_folder)
    print(f"'{results_folder}' folder created successfully.")
except FileExistsError:
    print(f"'{results_folder}' folder already exists.")

# Save the df containing per_label results into a CSV file
final_df.to_csv(results_folder / f'{filename}_per_label_avg_int.csv')

'results\test_data\2D\Cellpose' folder created successfully.


<h3>Plot average intensities</h3>

In [7]:
# Select all column names in 'final_df' that contain the substring 'avg_int'
avg_int_columns = [col for col in final_df.columns if 'avg_int' in col]

# Loop over all extracted channel average intensities
for column_name in avg_int_columns:

    # Plot the average_intensity distribution in order to make an informed decision on the threshold
    for roi_name in roi_names:

        # Filter rows in final_df where ROI matches roi_name 
        filtered_df = final_df[(final_df["ROI"] == roi_name)]

        # Get the values of the 'label' column in filtered_df as a list
        avg_int_values = filtered_df[column_name]

        # Plot a histogram with 256 bins and a title indicating the column and ROI
        fig = px.histogram(avg_int_values, nbins=256, 
                           title=f"{column_name}_in_{roi_name}",
                           labels={'value': column_name, 'count': 'Frequency'},
                           range_x=[0, avg_int_values.max()])
        
        # Show the plot
        fig.show() 

<h3>Select cells positive for a marker based on average intensity</h3>

Define in <code>min_max_per_marker</code> the <code>marker</code> you want to use to define your cell populations of interest, the <code>min_max</code> range of avg_int and the <code>population</code> name.


In [8]:
# You can choose markers from the following list
markers

[('ki67', 0, 'nucleus'), ('neun', 1, 'nucleus'), ('calbindin', 2, 'cytoplasm')]

In [9]:
# Create a dictionary with min_max_avg_int for each channel
# Give the possibility to define populations for the same marker (i.e. neun high and neun low)
# max_values are set to 255 since the test input images are 8-bit, higher bit depths can result in higher max avg_int values

min_max_per_marker = [{"marker": "ki67", "min_max": (200,255), "population":"ki67"},
                      {"marker": "neun", "min_max": (50,115), "population":"neun_low"},
                      {"marker": "neun", "min_max": (115,255), "population":"neun_high"},
                      {"marker": "calbindin", "min_max": (65,255), "population":"calbindin"},]

In [10]:
# Create an empty list to store all stats extracted from each image
stats = []

for marker_analysis in min_max_per_marker:

    marker = marker_analysis["marker"]
    min_max_avg_int = marker_analysis["min_max"]
    population = marker_analysis["population"]

    # Retrieve the column name from which the avg_int values should be read
    for column in avg_int_columns:
        if marker in column:
            column_name = column

    for roi_name in roi_names:

        # Initialize an empty list to hold the extracted dataframes on a per channel basis
        props_list = []

        # Retrieve the first and second values (channel and location) of the corresponding tuple in markers
        for item in markers:
            if item[0] == marker:
                channel = item[1]
                location = item[2]
                break  # Stop searching once the marker is found

        # Read the nuclei predictions per ROI
        nuclei_labels = tifffile.imread(nuclei_preds_path / roi_name / f"{filename}.tiff")

        # Filter rows in final_df where ROI matches roi_name and column_name is within min < avg_int <= max values.
        filtered_df = final_df[(final_df["ROI"] == roi_name) & (final_df[column_name] > min_max_avg_int[0]) & (final_df[column_name] <= min_max_avg_int[1])]

        # Get the values of the 'label' column in filtered_df as a list
        label_values = filtered_df["label"].tolist()

        # Create a boolean mask where each element is True if the corresponding value in 'nuclei_labels' 
        # is found in 'label_values', and False otherwise
        mask = np.isin(nuclei_labels, label_values)

        # Use the mask to set values in 'nuclei_labels' that are not in 'label_values' to 0,
        # creating a new array 'filtered_labels' with only the specified values retained
        filtered_labels = np.where(mask, nuclei_labels, 0)

        viewer.add_labels(filtered_labels, name=f"{population}_+_in_{roi_name}")

        # Extract your information of interest
        total_nuclei = len(np.unique(nuclei_labels)) - 1
        marker_pos_nuclei = len(np.unique(filtered_labels)) - 1

        # Calculate "%_marker+_cells" and avoid division by zero errors
        try:
            perc_marker_pos_cells = (marker_pos_nuclei * 100) / total_nuclei
        except ZeroDivisionError:
            perc_marker_pos_cells = 0

        # Create a dictionary containing all extracted info per masked image
        stats_dict = {
                    "filename": filename,
                    "ROI": roi_name,
                    "based_on": column_name,
                    "marker": marker,
                    "population":population,
                    "marker_ch": channel,
                    "location": location,
                    "min_max_avg_int": min_max_avg_int,
                    "total_nuclei": total_nuclei,
                    "marker+_nuclei": marker_pos_nuclei,
                    "%_marker+_cells": perc_marker_pos_cells,
                    "slicing_factor": slicing_factor
                    }
        
        # Append the current data point to the stats_list
        stats.append(stats_dict)  

<h3>Save number of positive cells based on each marker</h3>

In [11]:
# Define output folder for results
results_folder = Path("results") / experiment_id / segmentation_type / model_name

# Create the necessary folder structure if it does not exist
try:
    os.makedirs(str(results_folder))
    print(f"Output folder created: {results_folder}")
except FileExistsError:
    print(f"Output folder already exists: {results_folder}")

# Transform into a dataframe to store it as .csv later
df = pd.DataFrame(stats)

# Define the .csv path
csv_path = results_folder / f"SP_marker_+_label_avg_int.csv"

# Append to the .csv with new data points each round
df.to_csv(csv_path, mode="a", index=False, header=not os.path.isfile(csv_path))

# Show the updated .csv 
csv_df = pd.read_csv(csv_path)

csv_df

Output folder already exists: results\test_data\2D\Cellpose


,filename,ROI,based_on,marker,population,marker_ch,location,min_max_avg_int,total_nuclei,marker+_nuclei,%_marker+_cells,slicing_factor
0,HI1_CONTRA_M8_S6_TR1,CA,nucleus_ki67_avg_int,ki67,ki67,0,nucleus,"(200, 255)",1270,10,0.787402,NaN
1,HI1_CONTRA_M8_S6_TR1,DG,nucleus_ki67_avg_int,ki67,ki67,0,nucleus,"(200, 255)",1584,65,4.103535,NaN
2,HI1_CONTRA_M8_S6_TR1,CA,nucleus_neun_avg_int,neun,neun_low,1,nucleus,"(50, 115)",1270,223,17.559055,NaN
3,HI1_CONTRA_M8_S6_TR1,DG,nucleus_neun_avg_int,neun,neun_low,1,nucleus,"(50, 115)",1584,891,56.250000,NaN
4,HI1_CONTRA_M8_S6_TR1,CA,nucleus_neun_avg_int,neun,neun_high,1,nucleus,"(115, 255)",1270,871,68.582677,NaN
5,HI1_CONTRA_M8_S6_TR1,DG,nucleus_neun_avg_int,neun,neun_high,1,nucleus,"(115, 255)",1584,363,22.916667,NaN
6,HI1_CONTRA_M8_S6_TR1,CA,cytoplasm_calbindin_avg_int,calbindin,calbindin,2,cytoplasm,"(65, 255)",1270,0,0.000000,NaN
7,HI1_CONTRA_M8_S6_TR1,DG,cytoplasm_calbindin_avg_int,calbindin,calbindin,2,cytoplasm,"(65, 255)",1584,12,0.757576,NaN
